# Calculate DMD Modes
***

This script calculates windowed DMD modes as described in the paper *Extracting Reproducible Time-Resolved Resting State Networks using Dynamic Mode Decomposition*, by Kunert-Graf, Eschenburg, Galas, Kutz, Rane and Brunton.

In [2]:
%load_ext watermark
%watermark -a "James Kunert-Graf" -d -v -m -p h5py,tqdm,numpy,sklearn,nibabel -g -u

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


James Kunert-Graf 
last updated: 2018-04-24 

CPython 2.7.12
IPython 5.6.0

h5py 2.7.0
tqdm 4.11.2
numpy 1.14.2
sklearn 0.18
nibabel 2.1.0

compiler   : GCC 5.4.0 20160609
system     : Linux
release    : 4.4.0-119-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit
Git hash   : 0be1fc485c9d84cbd56a683784f53454e09b00f1


In [3]:
import os
import h5py
from tqdm import tqdm_notebook as tqdm
import numpy as np
from sklearn.decomposition import FastICA
import nibabel as nib

#import 'DMD', which runs DMD on an input data array (with rows as variables, columns as timepoints)
#and outputs DMD modes along with the associated powers and frequencies
from DMD import DMD
#import 'binCortexData', which takes grayordinate DMD modes and bins them spatially for clearer visualization
#and ease of clustering
from hcp_dmd_utils import binCortexData,loadGrayord

%matplotlib inline

## Parameters and Filepaths
***
The cell below contains parameters for the execution of the algorithm, along with variables specifying which data to analyze

**Parameters**

-  *FRAME0S*: list of where each window should start (setting last argument to '4' means windows slide in increments of 4)
-  *nmodes*: number of modes to calculate in each window
-  *nframes*: length of each window, in number of frames

**Filepath Information**
The analysis in this notebook iterates through many scans, as specified by these variables.

The code assumes that data is located in a directory with a structure such as given by this example:
*<font color="#ff0000">/path/to/data/</font><font color="#44aa00">102513</font>/rfMRI\_<font color="#aa44ff">REST1</font>\_<font color="#3333ff">LR</font>\_Atlas.dtseries.nii*

The above path is specified by the following variables:
- <font color="#ff0000">datapath</font>: parent directory of data folders 
- <font color="#44aa00">ulist</font>: list of subject IDs to analyze
- <font color="#aa44ff">runlabels</font>: list of run labels to analyze (e.g. REST1, REST2)
- <font color="#3333ff">scandir</font>: list of scan directions to analyze (LR/RL)

In [4]:
#PARAMETERS
FRAME0S=np.arange(0,1200,4)
nmodes=8
nframes=32

#only keep windows which do not extend beyond the data matrix
FRAME0S=FRAME0S[(FRAME0S+nframes)<=1200]

#FILEPATHS
#datapath='./data/'
datapath='/hdd/HCP_DMD/HCP_DMD/data/'

#load list of 120 subjects we analyzed
u120=np.genfromtxt('u120.txt').astype(int).astype(str)
ulist=u120

#define which scans to iterate over:
scandirs=['LR','RL']
runlabels=['REST1','REST2']

## Running the Analysis and Computing DMD Results

In short, the code below loops through each of the scan files identified above and does the following:
-  loads the scan data into the array **Xr**
-  loops through each window of data and computes the DMD modes, along with associated powers and frequencies
-  keep only those modes with positive frequencies, dropping redundant complex conjugate modes
-  keep track of metadata for each mode such as which individual it comes from, which window number, etc.
-  spatially bin each of the DMD modes
-  save all of the above in a directory './DMD_results'

In [ ]:
#'runlabel' and 'scandir' define which set of data to analyze
# and 'ulist' defines the list of individuals to analyze
for runlabel in runlabels:
    for scandir in scandirs:
        fdir='./DMD_results/nmodes{:}_nframes{:}_{:}_{:}/'.format(nmodes,nframes,runlabel,scandir)
        for k,u in enumerate(ulist):
            if os.path.exists(fdir+'Modes_{:}.h5'.format(u)) is False:
                FT=[]
                PT=[]
                ux=[]
                jx=[]
                jno=[]
                Phi=[]
                fname=datapath+u+'/rfMRI_'+runlabel+'_'+scandir+'_Atlas.dtseries.nii'
                #only perform if the corresponding data file exists..
                if os.path.exists(fname) is True:
                    print '[{:} modes, {:} frames] {:}/{:} :'.format(nmodes,nframes,k+1,len(ulist))+fname
                    #load data
                    Xr=loadGrayord(fname)
                    #keep only the first 59,412 grayordinates, which are the cortical grayordinates
                    #indices above this represent subcortical voxels which are ignored in this analysis
                    Xr=Xr[:59412]
                    #iterate through windows
                    for j,frame0 in enumerate(tqdm(FRAME0S)):
                        #select window of data
                        X=Xr[:,frame0:(frame0+nframes)]
                        
                        #perform DMD on window of data, calculated 'nmodes' modes
                        phik,pt,ft=DMD(X,n=nmodes)
                        #only keep positive frequencies (drop conjugates)
                        phik=phik[:,ft>=0]
                        pt=pt[ft>=0]
                        ft=ft[ft>=0]

                        FT.append(ft)
                        PT.append(pt)
                        Phi.append(phik)
                        #record:
                        #subject number (ux),
                        #window number (jx),
                        #mode number within window (jno)
                        ux.append(k*np.ones((len(ft),)))
                        jx.append(j*np.ones((len(ft),)))
                        jno.append(np.arange(len(ft)))
                        

                    #Separate collected modes into magnitude, phase
                    Phi=np.absolute(np.hstack(Phi))
                    Phase=np.angle(np.hstack(Phi))
                    #stack frequencies, powers, etc.
                    freq=np.hstack(FT)
                    power=np.hstack(PT)
                    ux=np.hstack(ux)
                    jx=np.hstack(jx)
                    jno=np.hstack(jno)

                    #% SAVE RESULTS INTO h5 FILE
                    try:
                        os.mkdir('./DMD_results')
                    except:
                        pass
                    try:
                        os.mkdir(fdir)
                    except:
                        pass
                    #save frequency, power, subject/window number, etc.
                    with h5py.File(fdir+'{:}.h5'.format(u),'w') as hf:
                        hf.create_dataset("freq",data=freq)
                        hf.create_dataset("power",data=power)
                        hf.create_dataset("jx",data=jx)
                        hf.create_dataset("jno",data=jno)
                        hf.create_dataset("ux",data=ux)
                        hf.create_dataset("nmodes",data=nmodes)
                        hf.create_dataset("nframes",data=nframes)
                        hf.create_dataset("scandir",data=scandir)
                    #% APPLY SPATIAL BINNING TO ALL MODES
                    F=binCortexData(Phi)
                    Th=binCortexData(Phase)
                    F=F.T
                    Th=Th.T
                    #save binned magnitude/phase data in separate file
                    with h5py.File(fdir+'Modes_{:}.h5'.format(u),'w') as hf:
                        hf.create_dataset("F",data=F)
                        hf.create_dataset("T",data=Th)
                else:
                    print('File does not exist: {:}'.format(fname))
                    
            else:
                print('{:} Already processed! Moving on...'.format(u))